# 解释器模式

FX 中一个有用的代码组织模式是循环遍历 {class}`~torch.fx.Graph` 中的所有 {class}`~torch.fx.Node` 并执行它们。这可以用于一些事情，包括对流经 {class}`~torch.fx.Graph` 的值的运行时分析，或者通过使用 {class}`~torch.fx.Proxy` 进行重跟踪的代码变换。例如，假设想要运行 {class}`~torch.fx。GraphModule` 并记录 {class}`~torch.Tensor` shape 和节点上的 dtype 属性，就像我们在运行时看到的那样。它可能看起来像:

In [ ]:
import torch
import torch.fx
from torch.fx.node import Node

from typing import Dict

class ShapeProp:
    """
    Shape propagation. This class takes a `GraphModule`.
    Then, its `propagate` method executes the `GraphModule`
    node-by-node with the given arguments. As each operation
    executes, the ShapeProp class stores away the shape and
    element type for the output values of each operation on
    the `shape` and `dtype` attributes of the operation's
    `Node`.
    """
    def __init__(self, mod):
        self.mod = mod
        self.graph = mod.graph
        self.modules = dict(self.mod.named_modules())

    def propagate(self, *args):
        args_iter = iter(args)
        env : Dict[str, Node] = {}

        def load_arg(a):
            return torch.fx.graph.map_arg(a, lambda n: env[n.name])

        def fetch_attr(target : str):
            target_atoms = target.split('.')
            attr_itr = self.mod
            for i, atom in enumerate(target_atoms):
                if not hasattr(attr_itr, atom):
                    raise RuntimeError(f"Node referenced nonexistant target {'.'.join(target_atoms[:i])}")
                attr_itr = getattr(attr_itr, atom)
            return attr_itr

        for node in self.graph.nodes:
            if node.op == 'placeholder':
                result = next(args_iter)
            elif node.op == 'get_attr':
                result = fetch_attr(node.target)
            elif node.op == 'call_function':
                result = node.target(*load_arg(node.args), **load_arg(node.kwargs))
            elif node.op == 'call_method':
                self_obj, *args = load_arg(node.args)
                kwargs = load_arg(node.kwargs)
                result = getattr(self_obj, node.target)(*args, **kwargs)
            elif node.op == 'call_module':
                result = self.modules[node.target](*load_arg(node.args), **load_arg(node.kwargs))

            # This is the only code specific to shape propagation.
            # you can delete this `if` branch and this becomes
            # a generic GraphModule interpreter.
            if isinstance(result, torch.Tensor):
                node.shape = result.shape
                node.dtype = result.dtype

            env[node.name] = result

        return load_arg(self.graph.result)

正如您所看到的，一个完整的 FX 解释器（interpreter）并不复杂，但它可能非常有用。为了方便使用这种模式，提供了 {class}`torch.fx.Interpreter` 类，它以一种可以通过方法重写来重写解释器执行的某些方面的方式包含了上述逻辑。

除了执行运算之外，还可以通过解释器提供 {class}`torch.fx.Proxy` 值来生成新的 {class}`torch.fx.Graph`。类似地，提供 {class}`torch.fx.Transformer` 类来包含此模式。{class}`torch.fx.Transformer` 的行为类似于 {class}`torch.fx.Interpreter`，但不是调用 `run` 方法从模块中获取具体的输出值，而是调用 {meth}`torch.fx.Transformer.transform` 方法来返回新的 {class}`torch.fx.GraphModule`，它服从于作为覆盖方法安装的任何变换规则。